In [1]:
import sys
sys.path.append("/home/ubuntu/MultiModalDeepFake")
import nemo.collections.asr as nemo_asr 
import pandas as pd

/home/ubuntu/.local/lib/python3.8/site-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.24.3) or chardet (5.1.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
2023-06-30 08:58:24.367123: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-30 08:58:24.479941: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-30 08:58:25.040038: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.c

In [2]:
from packages.LJDataLoader import LJDataLoader
from packages.AudioEmbeddingsManager import AudioEmbeddingsManager
from packages.ModelManager import ModelManager
from packages.CadenceModelManager import CadenceModelManager
import packages.AnalysisManager as am
from packages.SmileFeatureManager import SmileFeatureManager

In [3]:
def generate_split(fake_cols, file_path):

    
    loader = LJDataLoader(data_path=file_path, filter_cols=['ElevenLabsCloneClip'])
    loader.splitData()

    source_architectures = ['Full_Band_MelGan', 'HifiGan', 'MelGan', 'MelGanLarge', 'Multi_Band_MelGan', 'Parallel_WaveGan', 'Waveglow']
    new_col_name = 'RandWaveFake'
    loader.selectRandomArchitecture(target_col=new_col_name, source_cols=source_architectures)
    
    source_architectures = ['ElevenLabs', 'UberDuck']
    new_col_name = 'EL_UD_Fake'
    loader.selectRandomArchitecture(target_col=new_col_name, source_cols=source_architectures)
    
    source_architectures = ['RandWaveFake', 'ElevenLabs', 'UberDuck']
    new_col_name = 'Fake'
    loader.selectRandomArchitecture(target_col=new_col_name, source_cols=source_architectures)

    #data_df = loader.generateFinalDataFrame(real_col='Real', fake_cols=['RandWaveFake', 'ElevenLabs', 'UberDuck'])
    data_df = loader.generateFinalDataFrame(real_col='Real', fake_cols=fake_cols)
    
    train_count = data_df[data_df['type'] == 'train'].shape[0]
    dev_count = data_df[data_df['type'] == 'dev'].shape[0]
    test_count = data_df[data_df['type'] == 'test'].shape[0]

    print(f'# of Train instances: {train_count}')
    print(f'# of Dev instances: {dev_count}')
    print(f'# of Test instances: {test_count}')
    
    return data_df

In [4]:
df = generate_split(['ElevenLabs', 'UberDuck', 'Full_Band_MelGan', 'HifiGan', 'MelGan', 'MelGanLarge', 'Multi_Band_MelGan', 'Parallel_WaveGan', 'Waveglow'], '/home/ubuntu/data/wavefake_data/LJ_metadata_16000KHz.csv')
df

# of Train instances: 50260
# of Dev instances: 16760
# of Test instances: 16760


,type,id,architecture,path,label,multiclass_label
0,train,LJ018-0268,Real,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0
1,train,LJ041-0014,Real,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0
2,train,LJ031-0046,Real,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0
3,train,LJ016-0028,Real,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0
4,train,LJ030-0091,Real,/home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...,0,0
...,...,...,...,...,...,...
83775,test,LJ016-0285,Waveglow,/home/ubuntu/data/wavefake_data/generated_audi...,1,9
83776,test,LJ031-0190,Waveglow,/home/ubuntu/data/wavefake_data/generated_audi...,1,9
83777,test,LJ047-0077,Waveglow,/home/ubuntu/data/wavefake_data/generated_audi...,1,9
83778,test,LJ030-0056,Waveglow,/home/ubuntu/data/wavefake_data/generated_audi...,1,9


In [5]:
df[['architecture', 'label']].value_counts()

architecture       label
ElevenLabs         1        8378
Full_Band_MelGan   1        8378
HifiGan            1        8378
MelGan             1        8378
MelGanLarge        1        8378
Multi_Band_MelGan  1        8378
Parallel_WaveGan   1        8378
Real               0        8378
UberDuck           1        8378
Waveglow           1        8378
dtype: int64

In [6]:
def generate_features(data_df):
    speaker_model = nemo_asr.models.EncDecSpeakerLabelModel.from_pretrained(model_name='titanet_large')
    embedding_manager = AudioEmbeddingsManager(model=speaker_model, data=data_df)
    em_feature_df, em_feature_cols = embedding_manager.generateFeatureDf()
    
    cadence_manager = CadenceModelManager(data_df)
    cad_feature_df, cad_feature_cols, scalar =  cadence_manager.run_cadence_feature_extraction_pipeline(fill_na=-1) # Add param for load features or not
    
    smile_manager = SmileFeatureManager(data_df)
    #change number of features (feature_count=10 default)
    os_binary_feature_df, os_binary_feature_cols = smile_manager.generateFeatureDf('random_forest', label_type='binary', feature_count=20)
    os_multiclass_feature_df, os_multiclass_feature_cols = smile_manager.generateFeatureDf('random_forest', label_type='multiclass', feature_count=20)
    
    feature_store = {}
    feature_store['titanet'] = (em_feature_df, em_feature_cols)
    feature_store['openSmile_binary'] = (os_binary_feature_df, os_binary_feature_cols)
    feature_store['openSmile_multiclass'] = (os_multiclass_feature_df, os_multiclass_feature_cols)
    feature_store['cadence'] = (cad_feature_df, cad_feature_cols)
    
    return feature_store
    
    

In [7]:
def train_eval(feature_store, fake_cols):
    results_cols = ['feature_method', 'model', 'fake_cols', 'label_type', 'acc', 'cls_acc', 'loss', 'eer_score', 'eer_threshold']
    results = pd.DataFrame(columns=results_cols)
    
    models = ['logreg', 'random_forest']
    for model in models:
        for label_type in ['label', 'multiclass_label']:
            for k, v in feature_store.items():
                model_manager = ModelManager(model, v[0], v[1], merge_train_dev=True)
                model_manager.trainPredict(label_col=label_type)
                results = results.append(pd.DataFrame({'feature_method':[k], 'label_type':[label_type], 'fake_cols':[fake_cols], 'acc':[model_manager.accuracy], 'cls_acc':[model_manager.class_accuracy],  'loss':[model_manager.log_loss_value], 'model':[model], 
                                                       'eer_score':[model_manager.eer_score], 'eer_threshold':[model_manager.eer_threshold]}))
    
    return results
    
    

In [8]:
def run(fake_cols, metadata_path, name, data_df=None):
    if data_df is None:
        data_df = generate_split(fake_cols, metadata_path)
    feature_store = generate_features(data_df)
    results = train_eval(feature_store, fake_cols)
    results.to_csv(f'/home/ubuntu/data/results/6-30-23/{name}.csv', index=False)
    return results
    
    
    

In [9]:
agg_df = pd.DataFrame(columns=['feature_method', 'model', 'fake_cols', 'label_type', 'acc', 'cls_acc', 'loss', 'laundered', 'eer_score', 'eer_threshold'])

In [10]:
file_path = '/home/ubuntu/data/wavefake_data/LJ_metadata_16000KHz.csv'
results = run(['ElevenLabs'], file_path, '16KHz_ElevenLabs')
results['laundered'] = 0
agg_df = agg_df.append(results)

# of Train instances: 10052
# of Dev instances: 3352
# of Test instances: 3352
[NeMo I 2023-06-30 08:58:28 cloud:56] Found existing object /home/ubuntu/.cache/torch/NeMo/NeMo_1.15.0/titanet-l/492c0ab8416139171dc18c21879a9e45/titanet-l.nemo.
[NeMo I 2023-06-30 08:58:28 cloud:62] Re-using file from: /home/ubuntu/.cache/torch/NeMo/NeMo_1.15.0/titanet-l/492c0ab8416139171dc18c21879a9e45/titanet-l.nemo
[NeMo I 2023-06-30 08:58:28 common:913] Instantiating model from pre-trained checkpoint


[NeMo W 2023-06-30 08:58:29 modelPT:156] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /manifests/combined_fisher_swbd_voxceleb12_librispeech/train.json
    sample_rate: 16000
    labels: null
    batch_size: 64
    shuffle: true
    time_length: 3
    is_tarred: false
    tarred_audio_filepaths: null
    tarred_shard_strategy: scatter
    augmentor:
      noise:
        manifest_path: /manifests/noise/rir_noise_manifest.json
        prob: 0.5
        min_snr_db: 0
        max_snr_db: 15
      speed:
        prob: 0.5
        sr: 16000
        resample_type: kaiser_fast
        min_speed_rate: 0.95
        max_speed_rate: 1.05
    num_workers: 15
    pin_memory: true
    
[NeMo W 2023-06-30 08:58:29 modelPT:163] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_valida

[NeMo I 2023-06-30 08:58:29 label_models:126] Setting angular: true/false in decoder is deprecated and will be removed in 1.13 version, use specific loss with _target_
[NeMo I 2023-06-30 08:58:29 features:267] PADDING: 16
[NeMo I 2023-06-30 08:58:29 save_restore_connector:243] Model EncDecSpeakerLabelModel was successfully restored from /home/ubuntu/.cache/torch/NeMo/NeMo_1.15.0/titanet-l/492c0ab8416139171dc18c21879a9e45/titanet-l.nemo.


100%|██████████| 2/2 [00:26<00:00, 13.05s/it]


Initializing data...
Standardizing features...
smileFeatureSelectFromModel object initialized.


Selected features:.

1. audspecRasta_lengthL1norm_sma_qregc2
2. pcm_fftMag_spectralEntropy_sma_de_lpc4
3. pcm_fftMag_fband250-650_sma_de_peakMeanRel
4. audSpec_Rfilt_sma[4]_centroid
5. audSpec_Rfilt_sma[13]_centroid
6. pcm_fftMag_spectralFlux_sma_de_peakMeanRel
7. audspecRasta_lengthL1norm_sma_centroid
8. pcm_fftMag_spectralEntropy_sma_de_lpc0
9. pcm_fftMag_spectralEntropy_sma_upleveltime50
10. pcm_fftMag_spectralEntropy_sma_range
11. pcm_fftMag_spectralEntropy_sma_lpgain
12. pcm_fftMag_spectralEntropy_sma_de_maxPos
13. audSpec_Rfilt_sma[18]_centroid
14. pcm_RMSenergy_sma_minPos
15. pcm_fftMag_spectralEntropy_sma_linregerrQ
16. pcm_fftMag_spectralEntropy_sma_de_range
17. pcm_fftMag_spectralEntropy_sma_de_minPos
18. pcm_fftMag_spectralEntropy_sma_de_lpgain
19. pcm_fftMag_spectralEntropy_sma_skewness
20. audSpec_Rfilt_sma[5]_centroid
Initializing data...
Standardizing features...
smileFeature

[NeMo W 2023-06-30 09:00:44 nemo_logging:349] /tmp/ipykernel_4050536/903261330.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
      results = results.append(pd.DataFrame({'feature_method':[k], 'label_type':[label_type], 'fake_cols':[fake_cols], 'acc':[model_manager.accuracy], 'cls_acc':[model_manager.class_accuracy],  'loss':[model_manager.log_loss_value], 'model':[model],
    
[NeMo W 2023-06-30 09:00:44 nemo_logging:349] /home/ubuntu/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
    STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.
    
    Increase the number of iterations (max_iter) or scale the data as shown in:
        https://scikit-learn.org/stable/modules/preprocessing.html
    Please also refer to the documentation for alternative solver options:
        https://scikit-learn.org/stable/modules/linear_model

In [11]:
## SB_Comment - where are the final uned cadence results? I can see the params used below but dont think they're being used 
## SB_Comment - should set the window size and threshold as global params e.g. uberduck_window_size, elevenlabs_window_size, general_window_size etc 

In [12]:
file_path = '/home/ubuntu/data/wavefake_data/LJ_metadata_16000KHz.csv'
results = run(['UberDuck'], file_path, '16KHz_UberDuck')
results['laundered'] = 0
agg_df = agg_df.append(results)


# of Train instances: 10052
# of Dev instances: 3352
# of Test instances: 3352
[NeMo I 2023-06-30 09:01:25 cloud:56] Found existing object /home/ubuntu/.cache/torch/NeMo/NeMo_1.15.0/titanet-l/492c0ab8416139171dc18c21879a9e45/titanet-l.nemo.
[NeMo I 2023-06-30 09:01:25 cloud:62] Re-using file from: /home/ubuntu/.cache/torch/NeMo/NeMo_1.15.0/titanet-l/492c0ab8416139171dc18c21879a9e45/titanet-l.nemo
[NeMo I 2023-06-30 09:01:25 common:913] Instantiating model from pre-trained checkpoint


[NeMo W 2023-06-30 09:01:26 modelPT:156] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /manifests/combined_fisher_swbd_voxceleb12_librispeech/train.json
    sample_rate: 16000
    labels: null
    batch_size: 64
    shuffle: true
    time_length: 3
    is_tarred: false
    tarred_audio_filepaths: null
    tarred_shard_strategy: scatter
    augmentor:
      noise:
        manifest_path: /manifests/noise/rir_noise_manifest.json
        prob: 0.5
        min_snr_db: 0
        max_snr_db: 15
      speed:
        prob: 0.5
        sr: 16000
        resample_type: kaiser_fast
        min_speed_rate: 0.95
        max_speed_rate: 1.05
    num_workers: 15
    pin_memory: true
    
[NeMo W 2023-06-30 09:01:26 modelPT:163] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_valida

[NeMo I 2023-06-30 09:01:26 label_models:126] Setting angular: true/false in decoder is deprecated and will be removed in 1.13 version, use specific loss with _target_
[NeMo I 2023-06-30 09:01:26 features:267] PADDING: 16
[NeMo I 2023-06-30 09:01:27 save_restore_connector:243] Model EncDecSpeakerLabelModel was successfully restored from /home/ubuntu/.cache/torch/NeMo/NeMo_1.15.0/titanet-l/492c0ab8416139171dc18c21879a9e45/titanet-l.nemo.


100%|██████████| 2/2 [00:26<00:00, 13.15s/it]


Initializing data...
Standardizing features...
smileFeatureSelectFromModel object initialized.


Selected features:.

1. mfcc_sma[5]_lpc2
2. mfcc_sma[14]_peakMeanAbs
3. mfcc_sma_de[8]_lpgain
4. mfcc_sma[14]_quartile2
5. mfcc_sma[13]_lpgain
6. mfcc_sma_de[12]_meanRisingSlope
7. mfcc_sma[10]_meanPeakDist
8. mfcc_sma[14]_flatness
9. mfcc_sma_de[12]_lpgain
10. mfcc_sma[9]_lpgain
11. mfcc_sma_de[13]_lpgain
12. mfcc_sma_de[14]_lpc2
13. mfcc_sma_de[14]_meanFallingSlope
14. mfcc_sma_de[11]_lpc2
15. mfcc_sma[8]_lpgain
16. mfcc_sma[11]_lpc2
17. mfcc_sma[14]_amean
18. mfcc_sma[10]_lpc2
19. mfcc_sma_de[9]_meanRisingSlope
20. mfcc_sma[11]_meanPeakDist
Initializing data...
Standardizing features...
smileFeatureSelectFromModel object initialized.


Selected features:.

1. mfcc_sma[5]_lpc2
2. mfcc_sma[14]_peakMeanAbs
3. mfcc_sma_de[8]_lpgain
4. mfcc_sma[14]_quartile2
5. mfcc_sma[13]_lpgain
6. mfcc_sma_de[12]_meanRisingSlope
7. mfcc_sma[10]_meanPeakDist
8. mfcc_sma[14]_flatness
9. mfcc_sma_de[12]_lpgai

[NeMo W 2023-06-30 09:04:37 nemo_logging:349] /tmp/ipykernel_4050536/903261330.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
      results = results.append(pd.DataFrame({'feature_method':[k], 'label_type':[label_type], 'fake_cols':[fake_cols], 'acc':[model_manager.accuracy], 'cls_acc':[model_manager.class_accuracy],  'loss':[model_manager.log_loss_value], 'model':[model],
    
[NeMo W 2023-06-30 09:04:37 nemo_logging:349] /home/ubuntu/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
    STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.
    
    Increase the number of iterations (max_iter) or scale the data as shown in:
        https://scikit-learn.org/stable/modules/preprocessing.html
    Please also refer to the documentation for alternative solver options:
        https://scikit-learn.org/stable/modules/linear_model

In [13]:
file_path = '/home/ubuntu/data/wavefake_data/LJ_metadata_16000KHz.csv'
results = run(['RandWaveFake'], file_path, '16KHz_RandWaveFake_B')
results['laundered'] = 0
agg_df = agg_df.append(results)


# of Train instances: 10052
# of Dev instances: 3352
# of Test instances: 3352
[NeMo I 2023-06-30 09:05:38 cloud:56] Found existing object /home/ubuntu/.cache/torch/NeMo/NeMo_1.15.0/titanet-l/492c0ab8416139171dc18c21879a9e45/titanet-l.nemo.
[NeMo I 2023-06-30 09:05:38 cloud:62] Re-using file from: /home/ubuntu/.cache/torch/NeMo/NeMo_1.15.0/titanet-l/492c0ab8416139171dc18c21879a9e45/titanet-l.nemo
[NeMo I 2023-06-30 09:05:38 common:913] Instantiating model from pre-trained checkpoint


[NeMo W 2023-06-30 09:05:38 modelPT:156] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /manifests/combined_fisher_swbd_voxceleb12_librispeech/train.json
    sample_rate: 16000
    labels: null
    batch_size: 64
    shuffle: true
    time_length: 3
    is_tarred: false
    tarred_audio_filepaths: null
    tarred_shard_strategy: scatter
    augmentor:
      noise:
        manifest_path: /manifests/noise/rir_noise_manifest.json
        prob: 0.5
        min_snr_db: 0
        max_snr_db: 15
      speed:
        prob: 0.5
        sr: 16000
        resample_type: kaiser_fast
        min_speed_rate: 0.95
        max_speed_rate: 1.05
    num_workers: 15
    pin_memory: true
    
[NeMo W 2023-06-30 09:05:38 modelPT:163] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_valida

[NeMo I 2023-06-30 09:05:38 label_models:126] Setting angular: true/false in decoder is deprecated and will be removed in 1.13 version, use specific loss with _target_
[NeMo I 2023-06-30 09:05:38 features:267] PADDING: 16
[NeMo I 2023-06-30 09:05:39 save_restore_connector:243] Model EncDecSpeakerLabelModel was successfully restored from /home/ubuntu/.cache/torch/NeMo/NeMo_1.15.0/titanet-l/492c0ab8416139171dc18c21879a9e45/titanet-l.nemo.


100%|██████████| 8/8 [01:44<00:00, 13.09s/it]


Initializing data...
Standardizing features...
smileFeatureSelectFromModel object initialized.


Selected features:.

1. mfcc_sma[11]_lpgain
2. mfcc_sma[9]_lpc1
3. jitterDDP_sma_percentile1.0
4. jitterDDP_sma_iqr1-2
5. jitterLocal_sma_flatness
6. mfcc_sma_de[12]_lpgain
7. mfcc_sma_de[6]_lpc0
8. mfcc_sma_de[6]_lpc2
9. jitterLocal_sma_quartile1
10. mfcc_sma[14]_lpgain
11. mfcc_sma_de[13]_lpgain
12. jitterDDP_sma_quartile1
13. shimmerLocal_sma_quartile1
14. jitterDDP_sma_quartile2
15. mfcc_sma_de[5]_lpc1
16. jitterDDP_sma_flatness
17. mfcc_sma_de[11]_lpc2
18. jitterLocal_sma_percentile1.0
19. shimmerLocal_sma_percentile1.0
20. mfcc_sma_de[6]_lpc3
Initializing data...
Standardizing features...
smileFeatureSelectFromModel object initialized.


Selected features:.

1. mfcc_sma[11]_lpgain
2. mfcc_sma[9]_lpc1
3. jitterDDP_sma_percentile1.0
4. jitterDDP_sma_iqr1-2
5. jitterLocal_sma_flatness
6. mfcc_sma_de[12]_lpgain
7. mfcc_sma_de[6]_lpc0
8. mfcc_sma_de[6]_lpc2
9. jitterLocal_sma_quartile1
10.

[NeMo W 2023-06-30 09:10:14 nemo_logging:349] /tmp/ipykernel_4050536/903261330.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
      results = results.append(pd.DataFrame({'feature_method':[k], 'label_type':[label_type], 'fake_cols':[fake_cols], 'acc':[model_manager.accuracy], 'cls_acc':[model_manager.class_accuracy],  'loss':[model_manager.log_loss_value], 'model':[model],
    
[NeMo W 2023-06-30 09:10:15 nemo_logging:349] /home/ubuntu/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
    STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.
    
    Increase the number of iterations (max_iter) or scale the data as shown in:
        https://scikit-learn.org/stable/modules/preprocessing.html
    Please also refer to the documentation for alternative solver options:
        https://scikit-learn.org/stable/modules/linear_model

In [14]:
file_path = '/home/ubuntu/data/wavefake_data/LJ_metadata_16000KHz.csv'
results = run(['UberDuck', 'ElevenLabs'], file_path, '16KHz_ElevenLabs_and_UberDuck')
results['laundered'] = 0
agg_df = agg_df.append(results)


# of Train instances: 15078
# of Dev instances: 5028
# of Test instances: 5028
[NeMo I 2023-06-30 09:11:15 cloud:56] Found existing object /home/ubuntu/.cache/torch/NeMo/NeMo_1.15.0/titanet-l/492c0ab8416139171dc18c21879a9e45/titanet-l.nemo.
[NeMo I 2023-06-30 09:11:15 cloud:62] Re-using file from: /home/ubuntu/.cache/torch/NeMo/NeMo_1.15.0/titanet-l/492c0ab8416139171dc18c21879a9e45/titanet-l.nemo
[NeMo I 2023-06-30 09:11:15 common:913] Instantiating model from pre-trained checkpoint


[NeMo W 2023-06-30 09:11:16 modelPT:156] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /manifests/combined_fisher_swbd_voxceleb12_librispeech/train.json
    sample_rate: 16000
    labels: null
    batch_size: 64
    shuffle: true
    time_length: 3
    is_tarred: false
    tarred_audio_filepaths: null
    tarred_shard_strategy: scatter
    augmentor:
      noise:
        manifest_path: /manifests/noise/rir_noise_manifest.json
        prob: 0.5
        min_snr_db: 0
        max_snr_db: 15
      speed:
        prob: 0.5
        sr: 16000
        resample_type: kaiser_fast
        min_speed_rate: 0.95
        max_speed_rate: 1.05
    num_workers: 15
    pin_memory: true
    
[NeMo W 2023-06-30 09:11:16 modelPT:163] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_valida

[NeMo I 2023-06-30 09:11:16 label_models:126] Setting angular: true/false in decoder is deprecated and will be removed in 1.13 version, use specific loss with _target_
[NeMo I 2023-06-30 09:11:16 features:267] PADDING: 16
[NeMo I 2023-06-30 09:11:16 save_restore_connector:243] Model EncDecSpeakerLabelModel was successfully restored from /home/ubuntu/.cache/torch/NeMo/NeMo_1.15.0/titanet-l/492c0ab8416139171dc18c21879a9e45/titanet-l.nemo.


100%|██████████| 3/3 [00:40<00:00, 13.44s/it]


Initializing data...
Standardizing features...
smileFeatureSelectFromModel object initialized.


Selected features:.

1. mfcc_sma[5]_lpc2
2. mfcc_sma[14]_quartile3
3. mfcc_sma_de[8]_lpgain
4. mfcc_sma[14]_quartile2
5. mfcc_sma[13]_lpgain
6. mfcc_sma[14]_quartile1
7. mfcc_sma[10]_meanPeakDist
8. pcm_fftMag_spectralEntropy_sma_upleveltime50
9. pcm_fftMag_spectralEntropy_sma_range
10. mfcc_sma[9]_lpgain
11. pcm_fftMag_fband1000-4000_sma_quartile1
12. mfcc_sma[14]_rqmean
13. mfcc_sma_de[13]_lpgain
14. audspec_lengthL1norm_sma_de_peakMeanRel
15. pcm_fftMag_spectralEntropy_sma_de_minPos
16. mfcc_sma_de[11]_meanPeakDist
17. audspec_lengthL1norm_sma_rqmean
18. mfcc_sma[11]_lpc2
19. mfcc_sma[14]_amean
20. mfcc_sma[10]_lpc2
Initializing data...
Standardizing features...
smileFeatureSelectFromModel object initialized.


Selected features:.

1. mfcc_sma_de[8]_lpgain
2. pcm_fftMag_spectralEntropy_sma_de_lpc4
3. mfcc_sma[13]_lpgain
4. pcm_fftMag_spectralFlux_sma_de_peakMeanRel
5. pcm_fftMag_spectral

[NeMo W 2023-06-30 09:15:51 nemo_logging:349] /tmp/ipykernel_4050536/903261330.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
      results = results.append(pd.DataFrame({'feature_method':[k], 'label_type':[label_type], 'fake_cols':[fake_cols], 'acc':[model_manager.accuracy], 'cls_acc':[model_manager.class_accuracy],  'loss':[model_manager.log_loss_value], 'model':[model],
    
[NeMo W 2023-06-30 09:15:51 nemo_logging:349] /home/ubuntu/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
    STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.
    
    Increase the number of iterations (max_iter) or scale the data as shown in:
        https://scikit-learn.org/stable/modules/preprocessing.html
    Please also refer to the documentation for alternative solver options:
        https://scikit-learn.org/stable/modules/linear_model

In [15]:
file_path = '/home/ubuntu/data/wavefake_data/LJ_metadata_16000KHz.csv'
results = run(['ElevenLabs', 'UberDuck', 'RandWaveFake'], file_path, '16KHz_Mix')
results['laundered'] = 0
agg_df = agg_df.append(results)


# of Train instances: 20104
# of Dev instances: 6704
# of Test instances: 6704
[NeMo I 2023-06-30 09:17:23 cloud:56] Found existing object /home/ubuntu/.cache/torch/NeMo/NeMo_1.15.0/titanet-l/492c0ab8416139171dc18c21879a9e45/titanet-l.nemo.
[NeMo I 2023-06-30 09:17:23 cloud:62] Re-using file from: /home/ubuntu/.cache/torch/NeMo/NeMo_1.15.0/titanet-l/492c0ab8416139171dc18c21879a9e45/titanet-l.nemo
[NeMo I 2023-06-30 09:17:23 common:913] Instantiating model from pre-trained checkpoint


[NeMo W 2023-06-30 09:17:24 modelPT:156] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /manifests/combined_fisher_swbd_voxceleb12_librispeech/train.json
    sample_rate: 16000
    labels: null
    batch_size: 64
    shuffle: true
    time_length: 3
    is_tarred: false
    tarred_audio_filepaths: null
    tarred_shard_strategy: scatter
    augmentor:
      noise:
        manifest_path: /manifests/noise/rir_noise_manifest.json
        prob: 0.5
        min_snr_db: 0
        max_snr_db: 15
      speed:
        prob: 0.5
        sr: 16000
        resample_type: kaiser_fast
        min_speed_rate: 0.95
        max_speed_rate: 1.05
    num_workers: 15
    pin_memory: true
    
[NeMo W 2023-06-30 09:17:24 modelPT:163] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_valida

[NeMo I 2023-06-30 09:17:24 label_models:126] Setting angular: true/false in decoder is deprecated and will be removed in 1.13 version, use specific loss with _target_
[NeMo I 2023-06-30 09:17:24 features:267] PADDING: 16
[NeMo I 2023-06-30 09:17:25 save_restore_connector:243] Model EncDecSpeakerLabelModel was successfully restored from /home/ubuntu/.cache/torch/NeMo/NeMo_1.15.0/titanet-l/492c0ab8416139171dc18c21879a9e45/titanet-l.nemo.


100%|██████████| 10/10 [02:17<00:00, 13.76s/it]


Initializing data...
Standardizing features...
smileFeatureSelectFromModel object initialized.


Selected features:.

1. mfcc_sma_de[8]_lpgain
2. mfcc_sma[12]_lpgain
3. mfcc_sma[14]_quartile2
4. mfcc_sma[13]_lpgain
5. jitterDDP_sma_percentile1.0
6. mfcc_sma_de[9]_lpgain
7. mfcc_sma_de[12]_lpgain
8. pcm_fftMag_spectralEntropy_sma_upleveltime50
9. mfcc_sma[9]_lpgain
10. mfcc_sma[14]_lpgain
11. mfcc_sma_de[13]_lpgain
12. jitterDDP_sma_quartile1
13. jitterDDP_sma_quartile2
14. jitterDDP_sma_flatness
15. audspec_lengthL1norm_sma_amean
16. jitterLocal_sma_percentile1.0
17. mfcc_sma[7]_lpc0
18. audspec_lengthL1norm_sma_rqmean
19. mfcc_sma[11]_lpc2
20. mfcc_sma[14]_amean
Initializing data...
Standardizing features...
smileFeatureSelectFromModel object initialized.


Selected features:.

1. mfcc_sma[5]_lpc2
2. pcm_fftMag_spectralEntropy_sma_de_lpc4
3. pcm_fftMag_spectralFlux_sma_de_peakMeanRel
4. jitterDDP_sma_percentile1.0
5. pcm_fftMag_spectralEntropy_sma_de_minRangeRel
6. pcm_fftMag_spectral

[NeMo W 2023-06-30 09:25:16 nemo_logging:349] /tmp/ipykernel_4050536/903261330.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
      results = results.append(pd.DataFrame({'feature_method':[k], 'label_type':[label_type], 'fake_cols':[fake_cols], 'acc':[model_manager.accuracy], 'cls_acc':[model_manager.class_accuracy],  'loss':[model_manager.log_loss_value], 'model':[model],
    
[NeMo W 2023-06-30 09:25:17 nemo_logging:349] /home/ubuntu/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
    STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.
    
    Increase the number of iterations (max_iter) or scale the data as shown in:
        https://scikit-learn.org/stable/modules/preprocessing.html
    Please also refer to the documentation for alternative solver options:
        https://scikit-learn.org/stable/modules/linear_model

In [16]:
file_path = '/home/ubuntu/data/wavefake_data/LJ_metadata_16000KHz.csv'
results = run(['EL_UD_Fake'], file_path, '16KHz_EL_UD_Fake_B')
results['laundered'] = 0
agg_df = agg_df.append(results)


# of Train instances: 10052
# of Dev instances: 3352
# of Test instances: 3352
[NeMo I 2023-06-30 09:27:24 cloud:56] Found existing object /home/ubuntu/.cache/torch/NeMo/NeMo_1.15.0/titanet-l/492c0ab8416139171dc18c21879a9e45/titanet-l.nemo.
[NeMo I 2023-06-30 09:27:24 cloud:62] Re-using file from: /home/ubuntu/.cache/torch/NeMo/NeMo_1.15.0/titanet-l/492c0ab8416139171dc18c21879a9e45/titanet-l.nemo
[NeMo I 2023-06-30 09:27:24 common:913] Instantiating model from pre-trained checkpoint


[NeMo W 2023-06-30 09:27:25 modelPT:156] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /manifests/combined_fisher_swbd_voxceleb12_librispeech/train.json
    sample_rate: 16000
    labels: null
    batch_size: 64
    shuffle: true
    time_length: 3
    is_tarred: false
    tarred_audio_filepaths: null
    tarred_shard_strategy: scatter
    augmentor:
      noise:
        manifest_path: /manifests/noise/rir_noise_manifest.json
        prob: 0.5
        min_snr_db: 0
        max_snr_db: 15
      speed:
        prob: 0.5
        sr: 16000
        resample_type: kaiser_fast
        min_speed_rate: 0.95
        max_speed_rate: 1.05
    num_workers: 15
    pin_memory: true
    
[NeMo W 2023-06-30 09:27:25 modelPT:163] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_valida

[NeMo I 2023-06-30 09:27:25 label_models:126] Setting angular: true/false in decoder is deprecated and will be removed in 1.13 version, use specific loss with _target_
[NeMo I 2023-06-30 09:27:25 features:267] PADDING: 16
[NeMo I 2023-06-30 09:27:26 save_restore_connector:243] Model EncDecSpeakerLabelModel was successfully restored from /home/ubuntu/.cache/torch/NeMo/NeMo_1.15.0/titanet-l/492c0ab8416139171dc18c21879a9e45/titanet-l.nemo.


100%|██████████| 3/3 [00:38<00:00, 12.92s/it]


Initializing data...
Standardizing features...
smileFeatureSelectFromModel object initialized.


Selected features:.

1. mfcc_sma[5]_lpc2
2. mfcc_sma_de[12]_meanPeakDist
3. mfcc_sma[14]_quartile2
4. mfcc_sma[13]_lpgain
5. mfcc_sma[14]_quartile1
6. pcm_fftMag_spectralEntropy_sma_upleveltime50
7. pcm_fftMag_spectralEntropy_sma_range
8. mfcc_sma[9]_lpgain
9. pcm_fftMag_fband1000-4000_sma_quartile1
10. pcm_RMSenergy_sma_minPos
11. mfcc_sma[14]_rqmean
12. pcm_fftMag_spectralEntropy_sma_de_minPos
13. mfcc_sma[8]_lpgain
14. audspec_lengthL1norm_sma_rqmean
15. mfcc_sma[11]_lpc2
16. mfcc_sma[14]_amean
17. mfcc_sma[10]_lpc2
18. mfcc_sma_de[9]_meanRisingSlope
19. pcm_RMSenergy_sma_de_peakMeanRel
20. pcm_fftMag_spectralEntropy_sma_skewness
Initializing data...
Standardizing features...
smileFeatureSelectFromModel object initialized.


Selected features:.

1. mfcc_sma[5]_lpc2
2. mfcc_sma_de[12]_meanPeakDist
3. mfcc_sma[14]_quartile2
4. mfcc_sma[13]_lpgain
5. mfcc_sma[14]_quartile1
6. pcm_fftMag_spe

[NeMo W 2023-06-30 09:30:43 nemo_logging:349] /tmp/ipykernel_4050536/903261330.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
      results = results.append(pd.DataFrame({'feature_method':[k], 'label_type':[label_type], 'fake_cols':[fake_cols], 'acc':[model_manager.accuracy], 'cls_acc':[model_manager.class_accuracy],  'loss':[model_manager.log_loss_value], 'model':[model],
    
[NeMo W 2023-06-30 09:30:43 nemo_logging:349] /home/ubuntu/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
    STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.
    
    Increase the number of iterations (max_iter) or scale the data as shown in:
        https://scikit-learn.org/stable/modules/preprocessing.html
    Please also refer to the documentation for alternative solver options:
        https://scikit-learn.org/stable/modules/linear_model

In [ ]:
file_path = '/home/ubuntu/data/wavefake_data/LJ_metadata_16000KHz.csv'
results = run(['Fake'], file_path, '16KHz_Fake')
results['laundered'] = 0
agg_df = agg_df.append(results)


# of Train instances: 10052
# of Dev instances: 3352
# of Test instances: 3352
[NeMo I 2023-06-30 09:31:39 cloud:56] Found existing object /home/ubuntu/.cache/torch/NeMo/NeMo_1.15.0/titanet-l/492c0ab8416139171dc18c21879a9e45/titanet-l.nemo.
[NeMo I 2023-06-30 09:31:39 cloud:62] Re-using file from: /home/ubuntu/.cache/torch/NeMo/NeMo_1.15.0/titanet-l/492c0ab8416139171dc18c21879a9e45/titanet-l.nemo
[NeMo I 2023-06-30 09:31:39 common:913] Instantiating model from pre-trained checkpoint


[NeMo W 2023-06-30 09:31:39 modelPT:156] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /manifests/combined_fisher_swbd_voxceleb12_librispeech/train.json
    sample_rate: 16000
    labels: null
    batch_size: 64
    shuffle: true
    time_length: 3
    is_tarred: false
    tarred_audio_filepaths: null
    tarred_shard_strategy: scatter
    augmentor:
      noise:
        manifest_path: /manifests/noise/rir_noise_manifest.json
        prob: 0.5
        min_snr_db: 0
        max_snr_db: 15
      speed:
        prob: 0.5
        sr: 16000
        resample_type: kaiser_fast
        min_speed_rate: 0.95
        max_speed_rate: 1.05
    num_workers: 15
    pin_memory: true
    
[NeMo W 2023-06-30 09:31:39 modelPT:163] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_valida

[NeMo I 2023-06-30 09:31:39 label_models:126] Setting angular: true/false in decoder is deprecated and will be removed in 1.13 version, use specific loss with _target_
[NeMo I 2023-06-30 09:31:39 features:267] PADDING: 16
[NeMo I 2023-06-30 09:31:40 save_restore_connector:243] Model EncDecSpeakerLabelModel was successfully restored from /home/ubuntu/.cache/torch/NeMo/NeMo_1.15.0/titanet-l/492c0ab8416139171dc18c21879a9e45/titanet-l.nemo.


100%|██████████| 10/10 [02:11<00:00, 13.17s/it]


Initializing data...
Standardizing features...
smileFeatureSelectFromModel object initialized.


Selected features:.

1. mfcc_sma_de[8]_lpgain
2. mfcc_sma[12]_lpgain
3. mfcc_sma[13]_lpgain
4. jitterDDP_sma_percentile1.0
5. pcm_fftMag_spectralEntropy_sma_upleveltime50
6. pcm_fftMag_spectralEntropy_sma_range
7. mfcc_sma[9]_lpgain
8. pcm_fftMag_fband1000-4000_sma_quartile1
9. pcm_RMSenergy_sma_minPos
10. mfcc_sma_de[13]_lpgain
11. jitterDDP_sma_quartile1
12. jitterDDP_sma_flatness
13. pcm_fftMag_spectralEntropy_sma_de_minPos
14. jitterLocal_sma_percentile1.0
15. pcm_fftMag_spectralEntropy_sma_de_lpgain
16. mfcc_sma[7]_lpc0
17. mfcc_sma[8]_lpgain
18. audspec_lengthL1norm_sma_rqmean
19. mfcc_sma[11]_lpc2
20. mfcc_sma[14]_amean
Initializing data...
Standardizing features...
smileFeatureSelectFromModel object initialized.


Selected features:.

1. mfcc_sma_de[8]_lpgain
2. mfcc_sma[12]_lpgain
3. mfcc_sma[13]_lpgain
4. jitterDDP_sma_percentile1.0
5. pcm_fftMag_spectralEntropy_sma_upleveltime50


[NeMo W 2023-06-30 09:37:05 nemo_logging:349] /tmp/ipykernel_4050536/903261330.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
      results = results.append(pd.DataFrame({'feature_method':[k], 'label_type':[label_type], 'fake_cols':[fake_cols], 'acc':[model_manager.accuracy], 'cls_acc':[model_manager.class_accuracy],  'loss':[model_manager.log_loss_value], 'model':[model],
    
[NeMo W 2023-06-30 09:37:05 nemo_logging:349] /home/ubuntu/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
    STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.
    
    Increase the number of iterations (max_iter) or scale the data as shown in:
        https://scikit-learn.org/stable/modules/preprocessing.html
    Please also refer to the documentation for alternative solver options:
        https://scikit-learn.org/stable/modules/linear_model

In [18]:
file_path = '/home/ubuntu/data/wavefake_data/LJ_metadata_16KHz_Laundered.csv'
results = run(['ElevenLabs'], file_path, '16KHz_ElevenLabs_Laundered')
results['laundered'] = 1
agg_df = agg_df.append(results)



Selected features:.

1. audSpec_Rfilt_sma[2]_qregc1
2. audspecRasta_lengthL1norm_sma_qregc2
3. audSpec_Rfilt_sma[3]_linregc1
4. audSpec_Rfilt_sma[4]_linregc1
5. pcm_fftMag_spectralFlux_sma_de_peakMeanRel
6. audspecRasta_lengthL1norm_sma_centroid
7. audspecRasta_lengthL1norm_sma_maxPos
8. pcm_fftMag_spectralSlope_sma_percentile99.0
9. audSpec_Rfilt_sma[5]_qregc2
10. audSpec_Rfilt_sma[6]_qregc2
11. audSpec_Rfilt_sma[2]_qregc2
12. pcm_RMSenergy_sma_minPos
13. pcm_fftMag_spectralSlope_sma_upleveltime90
14. audSpec_Rfilt_sma[3]_qregc1
15. audSpec_Rfilt_sma[1]_qregc2
16. audSpec_Rfilt_sma[5]_linregc1
17. pcm_fftMag_spectralSlope_sma_peakMeanAbs
18. pcm_fftMag_spectralSlope_sma_minRangeRel
19. audSpec_Rfilt_sma[2]_centroid
20. audSpec_Rfilt_sma[5]_centroid
Initializing data...
Standardizing features...
smileFeatureSelectFromModel object initialized.


Selected features:.

1. audSpec_Rfilt_sma[2]_qregc1
2. audspecRasta_lengthL1norm_sma_qregc2
3. audSpec_Rfilt_sma[3]_linregc1
4. audSpec_Rfilt_

[NeMo W 2023-06-30 09:41:01 nemo_logging:349] /tmp/ipykernel_4050536/903261330.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
      results = results.append(pd.DataFrame({'feature_method':[k], 'label_type':[label_type], 'fake_cols':[fake_cols], 'acc':[model_manager.accuracy], 'cls_acc':[model_manager.class_accuracy],  'loss':[model_manager.log_loss_value], 'model':[model],
    
[NeMo W 2023-06-30 09:41:01 nemo_logging:349] /home/ubuntu/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
    STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.
    
    Increase the number of iterations (max_iter) or scale the data as shown in:
        https://scikit-learn.org/stable/modules/preprocessing.html
    Please also refer to the documentation for alternative solver options:
        https://scikit-learn.org/stable/modules/linear_model

In [19]:
file_path = '/home/ubuntu/data/wavefake_data/LJ_metadata_16KHz_Laundered.csv'
results = run(['UberDuck'], file_path, '16KHz_UberDuck_Laundered')
results['laundered'] = 1
agg_df = agg_df.append(results)


# of Train instances: 10052
# of Dev instances: 3350
# of Test instances: 3352
[NeMo I 2023-06-30 09:41:54 cloud:56] Found existing object /home/ubuntu/.cache/torch/NeMo/NeMo_1.15.0/titanet-l/492c0ab8416139171dc18c21879a9e45/titanet-l.nemo.
[NeMo I 2023-06-30 09:41:54 cloud:62] Re-using file from: /home/ubuntu/.cache/torch/NeMo/NeMo_1.15.0/titanet-l/492c0ab8416139171dc18c21879a9e45/titanet-l.nemo
[NeMo I 2023-06-30 09:41:54 common:913] Instantiating model from pre-trained checkpoint


[NeMo W 2023-06-30 09:41:55 modelPT:156] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /manifests/combined_fisher_swbd_voxceleb12_librispeech/train.json
    sample_rate: 16000
    labels: null
    batch_size: 64
    shuffle: true
    time_length: 3
    is_tarred: false
    tarred_audio_filepaths: null
    tarred_shard_strategy: scatter
    augmentor:
      noise:
        manifest_path: /manifests/noise/rir_noise_manifest.json
        prob: 0.5
        min_snr_db: 0
        max_snr_db: 15
      speed:
        prob: 0.5
        sr: 16000
        resample_type: kaiser_fast
        min_speed_rate: 0.95
        max_speed_rate: 1.05
    num_workers: 15
    pin_memory: true
    
[NeMo W 2023-06-30 09:41:55 modelPT:163] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_valida

[NeMo I 2023-06-30 09:41:55 label_models:126] Setting angular: true/false in decoder is deprecated and will be removed in 1.13 version, use specific loss with _target_
[NeMo I 2023-06-30 09:41:55 features:267] PADDING: 16
[NeMo I 2023-06-30 09:41:56 save_restore_connector:243] Model EncDecSpeakerLabelModel was successfully restored from /home/ubuntu/.cache/torch/NeMo/NeMo_1.15.0/titanet-l/492c0ab8416139171dc18c21879a9e45/titanet-l.nemo.


100%|██████████| 2/2 [00:26<00:00, 13.41s/it]


Initializing data...
Standardizing features...
smileFeatureSelectFromModel object initialized.


Selected features:.

1. mfcc_sma[5]_lpc2
2. mfcc_sma[14]_peakMeanAbs
3. mfcc_sma[14]_quartile3
4. mfcc_sma[14]_quartile2
5. mfcc_sma[13]_lpgain
6. mfcc_sma_de[12]_meanRisingSlope
7. mfcc_sma[14]_quartile1
8. mfcc_sma[9]_lpgain
9. mfcc_sma[12]_meanPeakDist
10. mfcc_sma[14]_peakMeanRel
11. mfcc_sma[14]_rqmean
12. mfcc_sma_de[13]_lpgain
13. mfcc_sma_de[14]_meanFallingSlope
14. mfcc_sma[13]_amean
15. mfcc_sma_de[9]_iqr1-3
16. mfcc_sma[8]_lpgain
17. mfcc_sma[11]_lpc2
18. mfcc_sma[14]_amean
19. mfcc_sma_de[9]_meanRisingSlope
20. mfcc_sma[11]_meanPeakDist
Initializing data...
Standardizing features...
smileFeatureSelectFromModel object initialized.


Selected features:.

1. mfcc_sma[5]_lpc2
2. mfcc_sma[14]_peakMeanAbs
3. mfcc_sma[14]_quartile3
4. mfcc_sma[14]_quartile2
5. mfcc_sma[13]_lpgain
6. mfcc_sma_de[12]_meanRisingSlope
7. mfcc_sma[14]_quartile1
8. mfcc_sma[9]_lpgain
9. mfcc_sma[12]_meanPeak

[NeMo W 2023-06-30 09:44:57 nemo_logging:349] /tmp/ipykernel_4050536/903261330.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
      results = results.append(pd.DataFrame({'feature_method':[k], 'label_type':[label_type], 'fake_cols':[fake_cols], 'acc':[model_manager.accuracy], 'cls_acc':[model_manager.class_accuracy],  'loss':[model_manager.log_loss_value], 'model':[model],
    
[NeMo W 2023-06-30 09:44:58 nemo_logging:349] /home/ubuntu/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
    STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.
    
    Increase the number of iterations (max_iter) or scale the data as shown in:
        https://scikit-learn.org/stable/modules/preprocessing.html
    Please also refer to the documentation for alternative solver options:
        https://scikit-learn.org/stable/modules/linear_model

In [ ]:
file_path = '/home/ubuntu/data/wavefake_data/LJ_metadata_16KHz_Laundered.csv'
results = run(['RandWaveFake'], file_path, '16KHz_RandWaveFake_Laundered')
results['laundered'] = 1
agg_df = agg_df.append(results)


# of Train instances: 10052
# of Dev instances: 3350
# of Test instances: 3352
[NeMo I 2023-06-30 09:45:59 cloud:56] Found existing object /home/ubuntu/.cache/torch/NeMo/NeMo_1.15.0/titanet-l/492c0ab8416139171dc18c21879a9e45/titanet-l.nemo.
[NeMo I 2023-06-30 09:45:59 cloud:62] Re-using file from: /home/ubuntu/.cache/torch/NeMo/NeMo_1.15.0/titanet-l/492c0ab8416139171dc18c21879a9e45/titanet-l.nemo
[NeMo I 2023-06-30 09:45:59 common:913] Instantiating model from pre-trained checkpoint


[NeMo W 2023-06-30 09:46:00 modelPT:156] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /manifests/combined_fisher_swbd_voxceleb12_librispeech/train.json
    sample_rate: 16000
    labels: null
    batch_size: 64
    shuffle: true
    time_length: 3
    is_tarred: false
    tarred_audio_filepaths: null
    tarred_shard_strategy: scatter
    augmentor:
      noise:
        manifest_path: /manifests/noise/rir_noise_manifest.json
        prob: 0.5
        min_snr_db: 0
        max_snr_db: 15
      speed:
        prob: 0.5
        sr: 16000
        resample_type: kaiser_fast
        min_speed_rate: 0.95
        max_speed_rate: 1.05
    num_workers: 15
    pin_memory: true
    
[NeMo W 2023-06-30 09:46:00 modelPT:163] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_valida

[NeMo I 2023-06-30 09:46:00 label_models:126] Setting angular: true/false in decoder is deprecated and will be removed in 1.13 version, use specific loss with _target_
[NeMo I 2023-06-30 09:46:00 features:267] PADDING: 16
[NeMo I 2023-06-30 09:46:01 save_restore_connector:243] Model EncDecSpeakerLabelModel was successfully restored from /home/ubuntu/.cache/torch/NeMo/NeMo_1.15.0/titanet-l/492c0ab8416139171dc18c21879a9e45/titanet-l.nemo.


100%|██████████| 8/8 [01:49<00:00, 13.71s/it]


Initializing data...
Standardizing features...
smileFeatureSelectFromModel object initialized.


Selected features:.

1. mfcc_sma_de[6]_lpc1
2. mfcc_sma_de[3]_lpc3
3. mfcc_sma[9]_lpc1
4. mfcc_sma_de[8]_lpc2
5. mfcc_sma_de[7]_lpc2
6. jitterDDP_sma_percentile1.0
7. jitterDDP_sma_iqr1-2
8. jitterLocal_sma_flatness
9. mfcc_sma_de[6]_lpc0
10. mfcc_sma_de[6]_lpc2
11. jitterLocal_sma_quartile1
12. jitterDDP_sma_quartile1
13. mfcc_sma[6]_lpc1
14. jitterDDP_sma_quartile2
15. mfcc_sma_de[5]_lpc1
16. mfcc_sma[5]_lpc1
17. jitterDDP_sma_flatness
18. jitterLocal_sma_percentile1.0
19. shimmerLocal_sma_percentile1.0
20. mfcc_sma_de[6]_lpc3
Initializing data...
Standardizing features...
smileFeatureSelectFromModel object initialized.


Selected features:.

1. mfcc_sma_de[6]_lpc1
2. mfcc_sma_de[3]_lpc3
3. mfcc_sma[9]_lpc1
4. mfcc_sma_de[8]_lpc2
5. mfcc_sma_de[7]_lpc2
6. jitterDDP_sma_percentile1.0
7. jitterDDP_sma_iqr1-2
8. jitterLocal_sma_flatness
9. mfcc_sma_de[6]_lpc0
10. mfcc_sma_de[6]_lpc2
11. jitt

[NeMo W 2023-06-30 09:50:39 nemo_logging:349] /tmp/ipykernel_4050536/903261330.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
      results = results.append(pd.DataFrame({'feature_method':[k], 'label_type':[label_type], 'fake_cols':[fake_cols], 'acc':[model_manager.accuracy], 'cls_acc':[model_manager.class_accuracy],  'loss':[model_manager.log_loss_value], 'model':[model],
    
[NeMo W 2023-06-30 09:50:39 nemo_logging:349] /home/ubuntu/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
    STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.
    
    Increase the number of iterations (max_iter) or scale the data as shown in:
        https://scikit-learn.org/stable/modules/preprocessing.html
    Please also refer to the documentation for alternative solver options:
        https://scikit-learn.org/stable/modules/linear_model

In [21]:
file_path = '/home/ubuntu/data/wavefake_data/LJ_metadata_16KHz_Laundered.csv'
results = run(['UberDuck', 'ElevenLabs'], file_path, '16KHz_ElevenLabs_and_UberDuck_Laundered')
results['laundered'] = 1
agg_df = agg_df.append(results)



Selected features:.

1. mfcc_sma[14]_quartile3
2. audspecRasta_lengthL1norm_sma_qregc2
3. mfcc_sma[14]_quartile2
4. mfcc_sma[13]_lpgain
5. mfcc_sma_de[12]_meanRisingSlope
6. mfcc_sma[14]_quartile1
7. mfcc_sma[10]_meanPeakDist
8. audspecRasta_lengthL1norm_sma_maxPos
9. pcm_fftMag_spectralSlope_sma_percentile99.0
10. pcm_fftMag_fband1000-4000_sma_quartile1
11. mfcc_sma[14]_peakMeanRel
12. mfcc_sma[14]_rqmean
13. pcm_fftMag_spectralSlope_sma_upleveltime90
14. mfcc_sma_de[13]_lpgain
15. mfcc_sma[6]_lpc0
16. mfcc_sma[14]_percentile1.0
17. audspec_lengthL1norm_sma_rqmean
18. mfcc_sma[14]_amean
19. mfcc_sma_de[9]_meanRisingSlope
20. pcm_fftMag_fband1000-4000_sma_iqr2-3
Initializing data...
Standardizing features...
smileFeatureSelectFromModel object initialized.


Selected features:.

1. audSpec_Rfilt_sma[2]_qregc1
2. mfcc_sma[14]_peakMeanAbs
3. audspecRasta_lengthL1norm_sma_qregc2
4. mfcc_sma[14]_quartile2
5. mfcc_sma[13]_lpgain
6. audSpec_Rfilt_sma[4]_linregc1
7. audSpec_Rfilt_sma[4]_centr

[NeMo W 2023-06-30 09:56:30 nemo_logging:349] /tmp/ipykernel_4050536/903261330.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
      results = results.append(pd.DataFrame({'feature_method':[k], 'label_type':[label_type], 'fake_cols':[fake_cols], 'acc':[model_manager.accuracy], 'cls_acc':[model_manager.class_accuracy],  'loss':[model_manager.log_loss_value], 'model':[model],
    
[NeMo W 2023-06-30 09:56:30 nemo_logging:349] /home/ubuntu/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
    STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.
    
    Increase the number of iterations (max_iter) or scale the data as shown in:
        https://scikit-learn.org/stable/modules/preprocessing.html
    Please also refer to the documentation for alternative solver options:
        https://scikit-learn.org/stable/modules/linear_model

In [22]:
file_path = '/home/ubuntu/data/wavefake_data/LJ_metadata_16KHz_Laundered.csv'
results = run(['ElevenLabs', 'UberDuck', 'RandWaveFake'], file_path, '16KHz_Mix_Laundered')
results['laundered'] = 1
agg_df = agg_df.append(results)


# of Train instances: 20104
# of Dev instances: 6700
# of Test instances: 6704
[NeMo I 2023-06-30 09:58:05 cloud:56] Found existing object /home/ubuntu/.cache/torch/NeMo/NeMo_1.15.0/titanet-l/492c0ab8416139171dc18c21879a9e45/titanet-l.nemo.
[NeMo I 2023-06-30 09:58:05 cloud:62] Re-using file from: /home/ubuntu/.cache/torch/NeMo/NeMo_1.15.0/titanet-l/492c0ab8416139171dc18c21879a9e45/titanet-l.nemo
[NeMo I 2023-06-30 09:58:05 common:913] Instantiating model from pre-trained checkpoint


[NeMo W 2023-06-30 09:58:06 modelPT:156] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /manifests/combined_fisher_swbd_voxceleb12_librispeech/train.json
    sample_rate: 16000
    labels: null
    batch_size: 64
    shuffle: true
    time_length: 3
    is_tarred: false
    tarred_audio_filepaths: null
    tarred_shard_strategy: scatter
    augmentor:
      noise:
        manifest_path: /manifests/noise/rir_noise_manifest.json
        prob: 0.5
        min_snr_db: 0
        max_snr_db: 15
      speed:
        prob: 0.5
        sr: 16000
        resample_type: kaiser_fast
        min_speed_rate: 0.95
        max_speed_rate: 1.05
    num_workers: 15
    pin_memory: true
    
[NeMo W 2023-06-30 09:58:06 modelPT:163] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_valida

[NeMo I 2023-06-30 09:58:06 label_models:126] Setting angular: true/false in decoder is deprecated and will be removed in 1.13 version, use specific loss with _target_
[NeMo I 2023-06-30 09:58:06 features:267] PADDING: 16
[NeMo I 2023-06-30 09:58:07 save_restore_connector:243] Model EncDecSpeakerLabelModel was successfully restored from /home/ubuntu/.cache/torch/NeMo/NeMo_1.15.0/titanet-l/492c0ab8416139171dc18c21879a9e45/titanet-l.nemo.


100%|██████████| 10/10 [02:14<00:00, 13.41s/it]


Initializing data...
Standardizing features...
smileFeatureSelectFromModel object initialized.


Selected features:.

1. mfcc_sma[14]_quartile3
2. audspecRasta_lengthL1norm_sma_qregc2
3. mfcc_sma[13]_lpgain
4. mfcc_sma[14]_quartile1
5. jitterDDP_sma_percentile1.0
6. mfcc_sma_de[9]_lpgain
7. mfcc_sma_de[12]_lpgain
8. mfcc_sma_de[6]_lpc0
9. mfcc_sma[9]_lpgain
10. mfcc_sma[14]_rqmean
11. mfcc_sma_de[13]_lpgain
12. jitterDDP_sma_quartile1
13. mfcc_sma[6]_lpc0
14. jitterDDP_sma_quartile2
15. jitterDDP_sma_flatness
16. jitterLocal_sma_percentile1.0
17. mfcc_sma[7]_lpc0
18. mfcc_sma[8]_lpgain
19. mfcc_sma[14]_amean
20. pcm_fftMag_fband1000-4000_sma_iqr2-3
Initializing data...
Standardizing features...
smileFeatureSelectFromModel object initialized.


Selected features:.

1. mfcc_sma[14]_peakMeanAbs
2. audspecRasta_lengthL1norm_sma_qregc2
3. mfcc_sma[14]_quartile2
4. mfcc_sma[13]_lpgain
5. audSpec_Rfilt_sma[4]_linregc1
6. audSpec_Rfilt_sma[4]_centroid
7. pcm_fftMag_spectralFlux_sma_de_peakMean

[NeMo W 2023-06-30 10:06:18 nemo_logging:349] /tmp/ipykernel_4050536/903261330.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
      results = results.append(pd.DataFrame({'feature_method':[k], 'label_type':[label_type], 'fake_cols':[fake_cols], 'acc':[model_manager.accuracy], 'cls_acc':[model_manager.class_accuracy],  'loss':[model_manager.log_loss_value], 'model':[model],
    
[NeMo W 2023-06-30 10:06:19 nemo_logging:349] /home/ubuntu/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
    STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.
    
    Increase the number of iterations (max_iter) or scale the data as shown in:
        https://scikit-learn.org/stable/modules/preprocessing.html
    Please also refer to the documentation for alternative solver options:
        https://scikit-learn.org/stable/modules/linear_model

In [23]:
file_path = '/home/ubuntu/data/wavefake_data/LJ_metadata_16KHz_Laundered.csv'
results = run(['EL_UD_Fake'], file_path, '16KHz_EL_UD_Fake_Laundered_B')
results['laundered'] = 1
agg_df = agg_df.append(results)


# of Train instances: 10052
# of Dev instances: 3350
# of Test instances: 3352
[NeMo I 2023-06-30 10:08:33 cloud:56] Found existing object /home/ubuntu/.cache/torch/NeMo/NeMo_1.15.0/titanet-l/492c0ab8416139171dc18c21879a9e45/titanet-l.nemo.
[NeMo I 2023-06-30 10:08:33 cloud:62] Re-using file from: /home/ubuntu/.cache/torch/NeMo/NeMo_1.15.0/titanet-l/492c0ab8416139171dc18c21879a9e45/titanet-l.nemo
[NeMo I 2023-06-30 10:08:33 common:913] Instantiating model from pre-trained checkpoint


[NeMo W 2023-06-30 10:08:34 modelPT:156] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /manifests/combined_fisher_swbd_voxceleb12_librispeech/train.json
    sample_rate: 16000
    labels: null
    batch_size: 64
    shuffle: true
    time_length: 3
    is_tarred: false
    tarred_audio_filepaths: null
    tarred_shard_strategy: scatter
    augmentor:
      noise:
        manifest_path: /manifests/noise/rir_noise_manifest.json
        prob: 0.5
        min_snr_db: 0
        max_snr_db: 15
      speed:
        prob: 0.5
        sr: 16000
        resample_type: kaiser_fast
        min_speed_rate: 0.95
        max_speed_rate: 1.05
    num_workers: 15
    pin_memory: true
    
[NeMo W 2023-06-30 10:08:34 modelPT:163] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_valida

[NeMo I 2023-06-30 10:08:34 label_models:126] Setting angular: true/false in decoder is deprecated and will be removed in 1.13 version, use specific loss with _target_
[NeMo I 2023-06-30 10:08:34 features:267] PADDING: 16
[NeMo I 2023-06-30 10:08:34 save_restore_connector:243] Model EncDecSpeakerLabelModel was successfully restored from /home/ubuntu/.cache/torch/NeMo/NeMo_1.15.0/titanet-l/492c0ab8416139171dc18c21879a9e45/titanet-l.nemo.


100%|██████████| 3/3 [00:39<00:00, 13.32s/it]


Initializing data...
Standardizing features...
smileFeatureSelectFromModel object initialized.


Selected features:.

1. audSpec_Rfilt_sma[2]_qregc1
2. mfcc_sma[14]_peakMeanAbs
3. mfcc_sma[14]_quartile3
4. audspecRasta_lengthL1norm_sma_qregc2
5. mfcc_sma[14]_quartile2
6. pcm_fftMag_spectralFlux_sma_de_peakMeanRel
7. mfcc_sma[14]_quartile1
8. mfcc_sma[10]_meanPeakDist
9. audspecRasta_lengthL1norm_sma_maxPos
10. mfcc_sma[9]_lpgain
11. mfcc_sma[12]_meanPeakDist
12. mfcc_sma[14]_peakMeanRel
13. mfcc_sma[14]_rqmean
14. mfcc_sma[6]_lpc0
15. mfcc_sma[14]_percentile1.0
16. audSpec_Rfilt_sma[5]_linregc1
17. audspec_lengthL1norm_sma_rqmean
18. mfcc_sma[11]_lpc2
19. mfcc_sma[14]_amean
20. pcm_fftMag_spectralSlope_sma_minRangeRel
Initializing data...
Standardizing features...
smileFeatureSelectFromModel object initialized.


Selected features:.

1. audSpec_Rfilt_sma[2]_qregc1
2. mfcc_sma[14]_peakMeanAbs
3. mfcc_sma[14]_quartile3
4. audspecRasta_lengthL1norm_sma_qregc2
5. mfcc_sma[14]_quartile2
6. 

[NeMo W 2023-06-30 10:11:49 nemo_logging:349] /tmp/ipykernel_4050536/903261330.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
      results = results.append(pd.DataFrame({'feature_method':[k], 'label_type':[label_type], 'fake_cols':[fake_cols], 'acc':[model_manager.accuracy], 'cls_acc':[model_manager.class_accuracy],  'loss':[model_manager.log_loss_value], 'model':[model],
    
[NeMo W 2023-06-30 10:11:49 nemo_logging:349] /home/ubuntu/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
    STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.
    
    Increase the number of iterations (max_iter) or scale the data as shown in:
        https://scikit-learn.org/stable/modules/preprocessing.html
    Please also refer to the documentation for alternative solver options:
        https://scikit-learn.org/stable/modules/linear_model

In [24]:
file_path = '/home/ubuntu/data/wavefake_data/LJ_metadata_16KHz_Laundered.csv'
results = run(['Fake'], file_path, '16KHz_Fake_Laundered_B')
results['laundered'] = 1
agg_df = agg_df.append(results)


# of Train instances: 10052
# of Dev instances: 3350
# of Test instances: 3352
[NeMo I 2023-06-30 10:12:47 cloud:56] Found existing object /home/ubuntu/.cache/torch/NeMo/NeMo_1.15.0/titanet-l/492c0ab8416139171dc18c21879a9e45/titanet-l.nemo.
[NeMo I 2023-06-30 10:12:47 cloud:62] Re-using file from: /home/ubuntu/.cache/torch/NeMo/NeMo_1.15.0/titanet-l/492c0ab8416139171dc18c21879a9e45/titanet-l.nemo
[NeMo I 2023-06-30 10:12:47 common:913] Instantiating model from pre-trained checkpoint


[NeMo W 2023-06-30 10:12:47 modelPT:156] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /manifests/combined_fisher_swbd_voxceleb12_librispeech/train.json
    sample_rate: 16000
    labels: null
    batch_size: 64
    shuffle: true
    time_length: 3
    is_tarred: false
    tarred_audio_filepaths: null
    tarred_shard_strategy: scatter
    augmentor:
      noise:
        manifest_path: /manifests/noise/rir_noise_manifest.json
        prob: 0.5
        min_snr_db: 0
        max_snr_db: 15
      speed:
        prob: 0.5
        sr: 16000
        resample_type: kaiser_fast
        min_speed_rate: 0.95
        max_speed_rate: 1.05
    num_workers: 15
    pin_memory: true
    
[NeMo W 2023-06-30 10:12:47 modelPT:163] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_valida

[NeMo I 2023-06-30 10:12:47 label_models:126] Setting angular: true/false in decoder is deprecated and will be removed in 1.13 version, use specific loss with _target_
[NeMo I 2023-06-30 10:12:47 features:267] PADDING: 16
[NeMo I 2023-06-30 10:12:48 save_restore_connector:243] Model EncDecSpeakerLabelModel was successfully restored from /home/ubuntu/.cache/torch/NeMo/NeMo_1.15.0/titanet-l/492c0ab8416139171dc18c21879a9e45/titanet-l.nemo.


100%|██████████| 10/10 [02:11<00:00, 13.13s/it]


Initializing data...
Standardizing features...
smileFeatureSelectFromModel object initialized.


Selected features:.

1. mfcc_sma[14]_quartile3
2. pcm_fftMag_fband1000-4000_sma_de_iqr1-3
3. mfcc_sma[14]_quartile1
4. jitterDDP_sma_percentile1.0
5. pcm_fftMag_spectralFlux_sma_pctlrange0-1
6. mfcc_sma_de[6]_lpc0
7. mfcc_sma[9]_lpgain
8. pcm_fftMag_fband1000-4000_sma_quartile1
9. mfcc_sma[14]_rqmean
10. mfcc_sma_de[13]_lpgain
11. pcm_fftMag_fband1000-4000_sma_quartile3
12. jitterDDP_sma_quartile1
13. mfcc_sma[6]_lpc0
14. jitterLocal_sma_percentile1.0
15. mfcc_sma[7]_lpc0
16. mfcc_sma[8]_lpgain
17. mfcc_sma[14]_percentile1.0
18. audspec_lengthL1norm_sma_rqmean
19. mfcc_sma[14]_amean
20. pcm_fftMag_fband1000-4000_sma_iqr2-3
Initializing data...
Standardizing features...
smileFeatureSelectFromModel object initialized.


Selected features:.

1. mfcc_sma[14]_quartile3
2. pcm_fftMag_fband1000-4000_sma_de_iqr1-3
3. mfcc_sma[14]_quartile1
4. jitterDDP_sma_percentile1.0
5. pcm_fftMag_spectralFlux_s

[NeMo W 2023-06-30 10:17:52 nemo_logging:349] /tmp/ipykernel_4050536/903261330.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
      results = results.append(pd.DataFrame({'feature_method':[k], 'label_type':[label_type], 'fake_cols':[fake_cols], 'acc':[model_manager.accuracy], 'cls_acc':[model_manager.class_accuracy],  'loss':[model_manager.log_loss_value], 'model':[model],
    
[NeMo W 2023-06-30 10:17:52 nemo_logging:349] /home/ubuntu/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
    STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.
    
    Increase the number of iterations (max_iter) or scale the data as shown in:
        https://scikit-learn.org/stable/modules/preprocessing.html
    Please also refer to the documentation for alternative solver options:
        https://scikit-learn.org/stable/modules/linear_model

In [25]:
agg_df.to_csv('/home/ubuntu/data/results/6-30-23/agg_results.csv', index=False)

In [26]:
import os
directory = '/home/ubuntu/data/TIMIT converted/wav_files'
file_count = len(list(os.listdir(directory)))
file_count

4620